# Convolutional Neural Networks (CNNs) Intro

This introduces convolutional layers to the basic neural network model. Usually you have a mix of convolutional layers with dense layers (or what PyTorch calls linear layers).

Typically, CNNs are used for image tasks. Although in recent years (current year being 2020), CNNs seems to be outperforming recurrent neural networks (RNNs) for handling sequential type of data.

In this intro, we will be looking at image data.

Unlike a typical dense layered NN, we can pass the image as is without having to flatten it. this includes 3D images.

The concept is that the convolutional layer will look for features in a kernel (a window of pixels from the image. for example, we could set the kernel to be a 3 pixel by 3 pixel for a 2D image of 100 pixels by 100 pixels). these features in the first convolutional layer could be edges, curves, and corners. then on the second layer, the features would be things that the patterns found on the first layer would build like circles, squares, etc. This process is called a convolution.

Once this is done on one kernel, the kernel slides through the image at set steps until it has gone through the entire image. ultimately condencing the image.

After the image is condenced, then pooling is done on the condenced data. the most common form of pooling is max pooling, which refers to extracting the maximum value of a kernel on the condenced data. Then as previously it done, the kernel will slide over the entire image until it has pooled the entire image.

